In [2]:
import os
import pandas as pd
import plotly.express as px
import progressbar
import json
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
pd.set_option('display.max_colwidth', None)

In [3]:
data_path = '../src/data'
mapping_corpus_path = data_path + r'/product/lspc2020_to_tablecorpus'
mapping_corpus_path_2 = data_path + r'/product/lspc2020_to_tablecorpus/Cleaned'
table_corpus_path = data_path + r'/product/product_top100/cleaned'
table_corpus_path_with_id = data_path + r'/product/product_top100/cleaned/with_id'
table_corpus_path2 = data_path + r'/product/product_minimum3/cleaned/with_id'

In [5]:
df_joined_electronics =  pd.read_csv(os.path.join(mapping_corpus_path_2, 'electronics_clusters_all_10_tables_post_processed.csv'))

In [18]:
df_electronics_final = pd.read_excel("Final_Electronics_v4.xlsx")

In [19]:
df_electronics_clusters = df_joined_electronics[df_joined_electronics['cluster_id'].isin(df_electronics_final['cluster_id'].to_list())]

In [20]:
df_electronics_clusters

,Unnamed: 0,Unnamed: 0.1,cluster_id,url,row_id,table_id,Valid,name,description,brand,tokens
0,9,11530,1001446,https://www.bts.com.au/store/p102/Apple_Lightning_Digital_AV_Adapter.html,90,Product_bts.com.au_September2020.json.gz,1,apple lightning digital av adapter,"Use the Lightning Digital AV Adapter with your iPhone, iPad or iPod with Lightning connector. The Lightning Digital AV Adapter supports mirroring of what is displayed on your device screen — including apps, presentations, websites, slideshows and more — to your HDMI-equipped TV, display, projector or other compatible display in up to 1080p HD.It also outputs video content — movies, TV shows, captured video — to your big screen in up to 1080p HD. Simply attach the Lightning Digital AV Adapter to the Lightning connector on your device and then to your TV or projector via an HDMI cable (sold separately).",NaN,"['apple', 'lightning', 'digital', 'adapter']"
1,10,11739,2859551,https://www.icentre.com.mt/apple-lightning-to-usb-cable-1m-2,511,Product_icentre.com.mt_September2020.json.gz,1,apple lightning to usb cable,"This USB 2.0 cable connects your iPhone, iPad, or iPod with Lightning connector to your computer’s USB port for syncing and charging. Or you can connect to the Apple USB Power Adapter for convenient charging from a wall outlet.",NaN,"['apple', 'lightning', 'usb', 'cable']"
2,14,18006,323989,https://store.macs-4-u.com.au/usb-c-charge-cable-1-m.html,192,Product_macs-4-u.com.au_September2020.json.gz,1,usb-c charge cable (1 m),USB-C Charge Cable (1 m),NaN,"['usb-c', 'charge', 'cable', '1']"
6,21,25661,2374515,https://www.tecnoprecios.net/tinta-original/860-multipack-canon-pg-545-cl-546-blister-mg2450-mg2550.html,611,Product_tecnoprecios.net_September2020.json.gz,1,cartucho tinta canon pg545 negro + cl546 color,MULTIPACK CANON PG-545/CL-546 BLISTER MG2450-MG2550,NaN,"['cartucho', 'tinta', 'canon', 'pg545', 'negro', 'cl546', 'color']"
9,29,30672,144625,https://www.cameraworld.co.uk/product/nikon-28-300mm-f3-5-5-6g-ed-vr-af-s-nikkor-lens-43805.html,764,Product_cameraworld.co.uk_September2020.json.gz,1,nikon 28-300mm f3.5-5.6g ed vr af-s nikkor lens,"F-Mount Lens/FX FormatAperture Range: f/3.5 to f/22Three Aspherical and Two ED ElementsSuper Integrated CoatingSilent Wave Motor AF SystemVR II Image StabilizationRounded 9-Blade DiaphragmZoom-lock switch keeps the lens secure when not in use.Weather-sealed mount protects the lens from dust and water.The AF-S NIKKOR 28-300mm f/3.5-5.6G ED VR’s huge 10.7 zoom ratio makes it the ultimate ‘walkabout’ lens. With an impressive 28-300mm focal range it is a very popular all-rounder for those seeking ultimate coverage from a single lens, useful when travelling light, or working from a fixed position.The flexible aperture range keeps the lens size and weight to a minimum while still providing attractive depth of field effects at the telephoto end.Advanced Vibration Reduction systemNikon’s exclusive second generation Vibration Reduction system (VRII) enables steady hand-held shooting at all focal lengths with two modes: Normal and Active.The optical VR system also stabilises the viewfinder image, enabling more accurate autofocus acquisition and framing, especially at the longer focal lengths of the 300mm zoom range, where lighter weight lenses can increase the risk of unwanted camera shake.State of the art optical constructionThe AF-S NIKKOR 28-300mm f/3.5-5.6G ED VR features two ED and three aspherical elements to effectively manage the risk of aberrations that can occur with super-zoom lenses.The lens can focus as close as 50cm at all focal lengths for added flexibility.Fast HandlingThe Silent Wave Motor (SWM) was designed specifically to deliver quiet and accurate AF operation across the extended zoom range with intuitive manual override. A zoom lock switch prevents ‘zoom creep’ (the lens moving under its own weight when tilted during transportation) and maintains compact dimensions when not in use.The lens construction features

In [21]:
# 2200 for around 9622 entries and 299 distinct cluster_ids
df_electronics_clusters.drop_duplicates(subset=['table_id'])

,Unnamed: 0,Unnamed: 0.1,cluster_id,url,row_id,table_id,Valid,name,description,brand,tokens
0,9,11530,1001446,https://www.bts.com.au/store/p102/Apple_Lightning_Digital_AV_Adapter.html,90,Product_bts.com.au_September2020.json.gz,1,apple lightning digital av adapter,"Use the Lightning Digital AV Adapter with your iPhone, iPad or iPod with Lightning connector. The Lightning Digital AV Adapter supports mirroring of what is displayed on your device screen — including apps, presentations, websites, slideshows and more — to your HDMI-equipped TV, display, projector or other compatible display in up to 1080p HD.It also outputs video content — movies, TV shows, captured video — to your big screen in up to 1080p HD. Simply attach the Lightning Digital AV Adapter to the Lightning connector on your device and then to your TV or projector via an HDMI cable (sold separately).",NaN,"['apple', 'lightning', 'digital', 'adapter']"
1,10,11739,2859551,https://www.icentre.com.mt/apple-lightning-to-usb-cable-1m-2,511,Product_icentre.com.mt_September2020.json.gz,1,apple lightning to usb cable,"This USB 2.0 cable connects your iPhone, iPad, or iPod with Lightning connector to your computer’s USB port for syncing and charging. Or you can connect to the Apple USB Power Adapter for convenient charging from a wall outlet.",NaN,"['apple', 'lightning', 'usb', 'cable']"
2,14,18006,323989,https://store.macs-4-u.com.au/usb-c-charge-cable-1-m.html,192,Product_macs-4-u.com.au_September2020.json.gz,1,usb-c charge cable (1 m),USB-C Charge Cable (1 m),NaN,"['usb-c', 'charge', 'cable', '1']"
6,21,25661,2374515,https://www.tecnoprecios.net/tinta-original/860-multipack-canon-pg-545-cl-546-blister-mg2450-mg2550.html,611,Product_tecnoprecios.net_September2020.json.gz,1,cartucho tinta canon pg545 negro + cl546 color,MULTIPACK CANON PG-545/CL-546 BLISTER MG2450-MG2550,NaN,"['cartucho', 'tinta', 'canon', 'pg545', 'negro', 'cl546', 'color']"
9,29,30672,144625,https://www.cameraworld.co.uk/product/nikon-28-300mm-f3-5-5-6g-ed-vr-af-s-nikkor-lens-43805.html,764,Product_cameraworld.co.uk_September2020.json.gz,1,nikon 28-300mm f3.5-5.6g ed vr af-s nikkor lens,"F-Mount Lens/FX FormatAperture Range: f/3.5 to f/22Three Aspherical and Two ED ElementsSuper Integrated CoatingSilent Wave Motor AF SystemVR II Image StabilizationRounded 9-Blade DiaphragmZoom-lock switch keeps the lens secure when not in use.Weather-sealed mount protects the lens from dust and water.The AF-S NIKKOR 28-300mm f/3.5-5.6G ED VR’s huge 10.7 zoom ratio makes it the ultimate ‘walkabout’ lens. With an impressive 28-300mm focal range it is a very popular all-rounder for those seeking ultimate coverage from a single lens, useful when travelling light, or working from a fixed position.The flexible aperture range keeps the lens size and weight to a minimum while still providing attractive depth of field effects at the telephoto end.Advanced Vibration Reduction systemNikon’s exclusive second generation Vibration Reduction system (VRII) enables steady hand-held shooting at all focal lengths with two modes: Normal and Active.The optical VR system also stabilises the viewfinder image, enabling more accurate autofocus acquisition and framing, especially at the longer focal lengths of the 300mm zoom range, where lighter weight lenses can increase the risk of unwanted camera shake.State of the art optical constructionThe AF-S NIKKOR 28-300mm f/3.5-5.6G ED VR features two ED and three aspherical elements to effectively manage the risk of aberrations that can occur with super-zoom lenses.The lens can focus as close as 50cm at all focal lengths for added flexibility.Fast HandlingThe Silent Wave Motor (SWM) was designed specifically to deliver quiet and accurate AF operation across the extended zoom range with intuitive manual override. A zoom lock switch prevents ‘zoom creep’ (the lens moving under its own weight when tilted during transportation) and maintains compact dimensions when not in use.The lens construction features

In [11]:
table_counts = df_electronics_clusters.groupby('table_id').count()

In [12]:
table_counts[table_counts ['cluster_id'].sort_values(ascending=False) > 15]

<ipython-input-12-a30fe86562f9>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  table_counts[table_counts ['cluster_id'].sort_values(ascending=False) > 15]


,Unnamed: 0,Unnamed: 0.1,cluster_id,url,row_id,Valid,name,description,brand,tokens
table_id,,,,,,,,,,
Product_allstars.com.sg_September2020.json.gz,19,19,19,19,19,19,19,19,0,19
Product_bellaphoto.com.br_September2020.json.gz,16,16,16,16,16,16,16,16,0,16
Product_bensonappliance.com_September2020.json.gz,20,20,20,20,20,20,20,0,0,20
Product_bilisimport.com.tr_September2020.json.gz,17,17,17,17,17,17,17,17,0,17
Product_bt.com_September2020.json.gz,26,26,26,26,26,26,26,26,0,26
Product_camerashopmuskegon.com_September2020.json.gz,16,16,16,16,16,16,16,16,0,16
Product_cclonline.com_September2020.json.gz,26,26,26,26,26,26,26,26,0,26
Product_crutchfield.com_September2020.json.gz,23,23,23,23,23,23,23,23,0,23
Product_diamondscamera.com.au_September2020.json.gz,30,30,30,30,30,30,30,30,30,30


In [13]:
df_electronics_clusters[['cluster_id','row_id','table_id','name']]

,cluster_id,row_id,table_id,name
0,1001446,90,Product_bts.com.au_September2020.json.gz,apple lightning digital av adapter
1,2859551,511,Product_icentre.com.mt_September2020.json.gz,apple lightning to usb cable
2,323989,192,Product_macs-4-u.com.au_September2020.json.gz,usb-c charge cable (1 m)
6,2374515,611,Product_tecnoprecios.net_September2020.json.gz,cartucho tinta canon pg545 negro + cl546 color
9,144625,764,Product_cameraworld.co.uk_September2020.json.gz,nikon 28-300mm f3.5-5.6g ed vr af-s nikkor lens
...,...,...,...,...
9312,3756053,110,Product_brandnameappliance.net_September2020.json.gz,23 cu. ft. counter depth 4-door french door refrigerator in stainless steel
9315,1929047,57,Product_kartmy.com_September2020.json.gz,samsung 970 evo 500gb nvme m.2 high speed solid state drive with v-nand
9318,32592,230,Product_copaco.com_September2020.json.gz,"western digital wd red pro 3.5"" 12000 gb serial ata iii"
9321,48051973,24,Product_ktrworld.com_September2020.json.gz,samsung gear vr virtual reality headset


In [42]:
df_cluster_323989 = df_electronics_clusters[df_electronics_clusters['cluster_id']==323989][['cluster_id','row_id','table_id','name']]

In [28]:
def jaccard_similarity_score(original, translation):
    intersect = set(original).intersection(set(translation))
    union = set(original).union(set(translation))
    try:
        return len(intersect) / len(union)
    except ZeroDivisionError:
        return 0

In [29]:
def remove_punctuation(token_vector):
    return token_vector.apply(lambda token_list: [word for word in token_list if word not in string.punctuation])

In [30]:
def remove_stopwords(token_vector, stopwords_list):
    return token_vector.apply(lambda token_list: [word for word in token_list if word not in stopwords_list])

In [47]:
df_cluster_323989['product_tokens'] = df_cluster_323989['name'].apply(lambda row: row.lower()).apply(lambda row: word_tokenize(row))
df_cluster_323989['product_tokens'] = remove_stopwords(df_cluster_323989['product_tokens'],stopwords.words())
df_cluster_323989['product_tokens'] = remove_punctuation (df_cluster_323989['product_tokens'])
df_cluster_323989.reset_index(inplace=True)

In [48]:
df_cluster_323989

,index,cluster_id,row_id,table_id,name,product_tokens
0,18006,323989,192,Product_macs-4-u.com.au_September2020.json.gz,USB-C Charge Cable (1 m),"[usb-c, charge, cable, 1]"
1,1318608,323989,1064,Product_retravision.com.au_September2020.json.gz,Apple USB-C Charge Cable (1M),"[apple, usb-c, charge, cable, 1m]"
2,2826897,323989,152,Product_pskmegastore.com_September2020.json.gz,Apple MUF72ZM/A cavo USB 1 m USB C Bianco,"[apple, muf72zm/a, cavo, usb, 1, usb, bianco]"
3,3207585,323989,9,Product_ilovecomputers.com.au_September2020.json.gz,Apple Apple USB-C Charge Cable (1 m),"[apple, apple, usb-c, charge, cable, 1]"
4,12580514,323989,146,Product_mcarerental.com_September2020.json.gz,Apple USB-C Charge Cable 1m,"[apple, usb-c, charge, cable, 1m]"
5,15121729,323989,321,Product_alza.co.uk_September2020.json.gz,Apple USB-C Charging Cable 1m,"[apple, usb-c, charging, cable, 1m]"
6,15290618,323989,20,Product_lecbroome.com.au_September2020.json.gz,USB-C Charge Cable (2M),"[usb-c, charge, cable, 2m]"
7,16279424,323989,8,Product_expotronica.com_September2020.json.gz,Apple cable conexión usb tipo c a usb-c 1m color blanco,"[apple, cable, conexión, usb, tipo, usb-c, 1m, color, blanco]"
8,17518365,323989,308,Product_gms-store.com_September2020.json.gz,Cabo USB-C para USB-C 1 metro,"[cabo, usb-c, usb-c, 1, metro]"
9,17912158,323989,5743,Product_euro.com.pl_September2020.json.gz,Apple Kabel USB-C,"[apple, kabel, usb-c]"


In [53]:
df_cluster_323989['jaccard_score'] = df_cluster_323989['product_tokens'].apply(lambda row: jaccard_similarity_score(row,df_cluster_323989.iloc[0]['product_tokens']) )

In [54]:
df_cluster_323989

,index,cluster_id,row_id,table_id,name,product_tokens,jaccard_score
0,18006,323989,192,Product_macs-4-u.com.au_September2020.json.gz,USB-C Charge Cable (1 m),"[usb-c, charge, cable, 1]",1.000000
1,1318608,323989,1064,Product_retravision.com.au_September2020.json.gz,Apple USB-C Charge Cable (1M),"[apple, usb-c, charge, cable, 1m]",0.500000
2,2826897,323989,152,Product_pskmegastore.com_September2020.json.gz,Apple MUF72ZM/A cavo USB 1 m USB C Bianco,"[apple, muf72zm/a, cavo, usb, 1, usb, bianco]",0.111111
3,3207585,323989,9,Product_ilovecomputers.com.au_September2020.json.gz,Apple Apple USB-C Charge Cable (1 m),"[apple, apple, usb-c, charge, cable, 1]",0.800000
4,12580514,323989,146,Product_mcarerental.com_September2020.json.gz,Apple USB-C Charge Cable 1m,"[apple, usb-c, charge, cable, 1m]",0.500000
5,15121729,323989,321,Product_alza.co.uk_September2020.json.gz,Apple USB-C Charging Cable 1m,"[apple, usb-c, charging, cable, 1m]",0.285714
6,15290618,323989,20,Product_lecbroome.com.au_September2020.json.gz,USB-C Charge Cable (2M),"[usb-c, charge, cable, 2m]",0.600000
7,16279424,323989,8,Product_expotronica.com_September2020.json.gz,Apple cable conexión usb tipo c a usb-c 1m color blanco,"[apple, cable, conexión, usb, tipo, usb-c, 1m, color, blanco]",0.181818
8,17518365,323989,308,Product_gms-store.com_September2020.json.gz,Cabo USB-C para USB-C 1 metro,"[cabo, usb-c, usb-c, 1, metro]",0.333333
9,17912158,323989,5743,Product_euro.com.pl_September2020.json.gz,Apple Kabel USB-C,"[apple, kabel, usb-c]",0.166667


In [22]:
df_joined_clothes = pd.read_csv(os.path.join(mapping_corpus_path_2, 'clothes_clusters_all_8_tables_post_processed.csv'))

In [23]:
df_clothes_final = pd.read_excel("Final_Clothes_v4.xlsx")

In [24]:
df_clothes_clusters = df_joined_clothes[df_joined_clothes['cluster_id'].isin(df_clothes_final['cluster_id'].to_list())]

In [26]:
df_clothes_clusters

,Unnamed: 0,Unnamed: 0.1,cluster_id,url,row_id,table_id,Valid,name,description,brand_x,brand_y,tokens
0,4,15925,78110534,https://www.cn.forzieri.com/chn/product_view.asp?l=chn&c=chn&dept_id=18&sku=dg130320-027-00,3400,Product_forzieri.com_September2020.json.gz,1,"small \""devotion\"" bag","Leather handle, detachable shoulder strap, front plate with logo jewel heart, magnetic closure, inner flat pocket, leather inner, gold finish metal accessories, made in italy.Sensorial gift wrap available",Dolce & Gabbana żΰ,NaN,"['small', ""''"", 'devotion\\', ""''"", 'bag']"
15,28,73795,58592784,https://sneaksa.com/product/nike-air-force-1-mid,45,Product_sneaksa.com_September2020.json.gz,1,nike air force 1 mid,"Le basketball dans le parc, le barbecue du dimanche et le soleil. Le charme continue à opérer avec la Nike Air Force 1 ’07. Cette silhouette emblématique du basketball apporte un nouveau souffle à ses éléments les plus remarquables : un cuir impeccable et un coloris blanc intégral pour une allure exceptionnelle sur le terrain comme au quotidien.",NaN,NaN,"['nike', 'air', 'force', '1', 'mid']"
16,30,76085,58083280,https://gamarraecommerce.com/producto/air-force-1-low-white-07,46,Product_gamarraecommerce.com_September2020.json.gz,1,air force 1 low white '07,"CODE 315122-111 COLORWAY WHITE/WHITE RETAIL PRICE $100These Products are supplied by a premium shoe marketplace, which offers the most coveted and hard-to-find footwear in the world.SIZE GUIDE: US (MEN) / US (WOMEN) / EUR",NaN,NaN,"['air', 'force', '1', 'low', 'white', ""'07""]"
22,40,96870,22653830,https://www.fitnewsole.com/travis-scott-x-nike-air-force-1-low-cactus-jack-cn2405-900-new-sale,121,Product_fitnewsole.com_September2020.json.gz,1,"travis scott x nike air force 1 low \""cactus jack\"" cn2405-900 new sale","A light brown canvas heel is covered in various Cactus Jack Records branding. Other details include a light brown canvas Swoosh, wool heel tab, plaid tongue, pink side panels, and dark grey leather toe box. A buckle with brass hardware has also been added to the heel.",NaN,NaN,"['travis', 'scott', 'x', 'nike', 'air', 'force', '1', 'low', ""''"", 'cactus', 'jack\\', ""''"", 'cn2405-900', 'new']"
32,57,139940,66437123,https://www.solehello.com/product/nike-air-mx-97-worldwide-white-blue-fury-black-volt-for-sale,429,Product_solehello.com_September2020.json.gz,1,"nike air max 97 “worldwide\"" white/blue fury-black-volt for sale","Dressed in a White, Blue Fury, Black, and Volt color scheme. This edition of the Nike Air Max 97 features a White upper paired with Blue and Volt contrasting accents with a Crimson tag on the lateral and “Worldwide” text on the Black heel tab.",NaN,nike,"['nike', 'air', 'max', '97', '“', 'worldwide\\', ""''"", 'white/blue', 'fury-black-volt']"
...,...,...,...,...,...,...,...,...,...,...,...,...
6008,9704,24624991,70109,https://da.ulapop.com/products/31872_dkblk,74,Product_ulapop.com_September2020.json.gz,1,coach - woman clutch bag,Køn: Kvindetype: Kobling Materiale: læder Vigtigste fastgørelse: metallisk Skulderrem: aftagelig skulderrem Indvendig: organiseret indvendigt Indvendige lommer: 2 Udvendige lommer: 1 Bredde cm: 18 Højde cm: 10 Dybde cm: 4 Detaljer: applikationer Støvtaske inkluderet synligt logo,NaN,NaN,"['coach', 'woman', 'clutch', 'bag']"
6011,9707,24627376,1784229,https://www.thesoleline.com/online/nike-air-vapormax-2020-summit-white-for-sale,109,Product_thesoleline.com_September2020.json.gz,1,nike air vapormax 2020 “summit white” for sale,This offering of theVaporMax 2020features a Summit White with Pink and Blue knits covering its Flyknit upper constructed of recycled polyester yarns with FlyEase lacing tech. A marbleized heel atop its signature recycled full-length VaporMax Air unit completes the design.,NaN,nike,"['nike', 'air', 'vapormax', '2020', '“', 'summit', 'white', '”']"
6014,9710,24628836,1916084,https://www.anytimewatches.com/en/tissot-t063-610-11-038-00.html,40,Product_anytimewatches.com_September2020.json.gz,1,tissot 

In [27]:
# 2200 for around 9622 entries and 299 distinct cluster_ids
df_clothes_clusters.drop_duplicates(subset=['table_id'])

,Unnamed: 0,Unnamed: 0.1,cluster_id,url,row_id,table_id,Valid,name,description,brand_x,brand_y,tokens
0,4,15925,78110534,https://www.cn.forzieri.com/chn/product_view.asp?l=chn&c=chn&dept_id=18&sku=dg130320-027-00,3400,Product_forzieri.com_September2020.json.gz,1,"small \""devotion\"" bag","Leather handle, detachable shoulder strap, front plate with logo jewel heart, magnetic closure, inner flat pocket, leather inner, gold finish metal accessories, made in italy.Sensorial gift wrap available",Dolce & Gabbana żΰ,NaN,"['small', ""''"", 'devotion\\', ""''"", 'bag']"
15,28,73795,58592784,https://sneaksa.com/product/nike-air-force-1-mid,45,Product_sneaksa.com_September2020.json.gz,1,nike air force 1 mid,"Le basketball dans le parc, le barbecue du dimanche et le soleil. Le charme continue à opérer avec la Nike Air Force 1 ’07. Cette silhouette emblématique du basketball apporte un nouveau souffle à ses éléments les plus remarquables : un cuir impeccable et un coloris blanc intégral pour une allure exceptionnelle sur le terrain comme au quotidien.",NaN,NaN,"['nike', 'air', 'force', '1', 'mid']"
16,30,76085,58083280,https://gamarraecommerce.com/producto/air-force-1-low-white-07,46,Product_gamarraecommerce.com_September2020.json.gz,1,air force 1 low white '07,"CODE 315122-111 COLORWAY WHITE/WHITE RETAIL PRICE $100These Products are supplied by a premium shoe marketplace, which offers the most coveted and hard-to-find footwear in the world.SIZE GUIDE: US (MEN) / US (WOMEN) / EUR",NaN,NaN,"['air', 'force', '1', 'low', 'white', ""'07""]"
22,40,96870,22653830,https://www.fitnewsole.com/travis-scott-x-nike-air-force-1-low-cactus-jack-cn2405-900-new-sale,121,Product_fitnewsole.com_September2020.json.gz,1,"travis scott x nike air force 1 low \""cactus jack\"" cn2405-900 new sale","A light brown canvas heel is covered in various Cactus Jack Records branding. Other details include a light brown canvas Swoosh, wool heel tab, plaid tongue, pink side panels, and dark grey leather toe box. A buckle with brass hardware has also been added to the heel.",NaN,NaN,"['travis', 'scott', 'x', 'nike', 'air', 'force', '1', 'low', ""''"", 'cactus', 'jack\\', ""''"", 'cn2405-900', 'new']"
32,57,139940,66437123,https://www.solehello.com/product/nike-air-mx-97-worldwide-white-blue-fury-black-volt-for-sale,429,Product_solehello.com_September2020.json.gz,1,"nike air max 97 “worldwide\"" white/blue fury-black-volt for sale","Dressed in a White, Blue Fury, Black, and Volt color scheme. This edition of the Nike Air Max 97 features a White upper paired with Blue and Volt contrasting accents with a Crimson tag on the lateral and “Worldwide” text on the Black heel tab.",NaN,nike,"['nike', 'air', 'max', '97', '“', 'worldwide\\', ""''"", 'white/blue', 'fury-black-volt']"
...,...,...,...,...,...,...,...,...,...,...,...,...
5926,9566,24272328,944406,https://www.thefragrancecounter.co.uk/dolce-and-gabbana-pour-homme-edt-75ml.html,314,Product_thefragrancecounter.co.uk_September2020.json.gz,1,dolce and gabbana pour homme edt 75ml,Buy cheap aftershave from Dolce and Gabbana on The Fragrance Counter. FREE Delivery. Many items with FREE gift with purchase. Rated 5* on Trustpilot.,Dolce and Gabbana,NaN,"['dolce', 'gabbana', 'homme', 'edt', '75ml']"
5934,9577,24313111,188731,https://www.vminnovations.com/Product_179028/Oakley-Standard-Fit-Holbrook-Sunglasses-Matte-Black-Prizm-Ruby-Open-Box-.html,937,Product_vminnovations.com_September2020.json.gz,1,"oakley standard fit holbrook sunglasses, matte black/prizm ruby (open box)","Item is open box, unharmed and in very good condition. All accessories or parts are included with the item.Classic design inspired by movie stars of the '40s, '50s, and '60s100% UV protection coating shields your eyes from the sun; great for people with lighter eye colors/light sensitivityFrame Color: Matte blackLens color: Prizm rubyFit: StandardLight Transmission: 17%Light Conditions: Bright lightContrast: IncreasedProduct Condition: Open BoxLike newProduct Wa

In [ ]:
df_clothes_clusters.drop_duplicates(subset=['table_id'])['table_id']

In [ ]:
table_counts = df_clothes_clusters.groupby('table_id').count()

In [ ]:
table_counts[table_counts ['cluster_id'].sort_values(ascending=False) > 15]

In [28]:
df_clothes_clusters[['cluster_id','row_id','table_id','name']]

,cluster_id,row_id,table_id,name
0,78110534,3400,Product_forzieri.com_September2020.json.gz,"small \""devotion\"" bag"
15,58592784,45,Product_sneaksa.com_September2020.json.gz,nike air force 1 mid
16,58083280,46,Product_gamarraecommerce.com_September2020.json.gz,air force 1 low white '07
22,22653830,121,Product_fitnewsole.com_September2020.json.gz,"travis scott x nike air force 1 low \""cactus jack\"" cn2405-900 new sale"
32,66437123,429,Product_solehello.com_September2020.json.gz,"nike air max 97 “worldwide\"" white/blue fury-black-volt for sale"
...,...,...,...,...
6008,70109,74,Product_ulapop.com_September2020.json.gz,coach - woman clutch bag
6011,1784229,109,Product_thesoleline.com_September2020.json.gz,nike air vapormax 2020 “summit white” for sale
6014,1916084,40,Product_anytimewatches.com_September2020.json.gz,tissot tradition t063.610.11.038.00
6015,1315657,123,Product_aufreshscents.com_September2020.json.gz,prada luna rossa sport by prada edt spray 3.4 oz
